In [ ]:
from numba import autojit, prange
import os
from pandas import DataFrame, read_csv, read_table
import time
from joblib import Parallel, delayed
from random import shuffle

In [ ]:
import os
import pickle
from pandas import read_csv, DataFrame, read_table
import numpy as np
from random import shuffle
from nltk.tokenize import TweetTokenizer, word_tokenize
from numba import autojit, prange

In [ ]:
CSV_PATH = "../Corpora/TO_CREATE_VALID_UTTERANCE"
EMOTIONS_PATH = CSV_PATH + "/emotions"
GOOGLE_NEWS_PATH = CSV_PATH + "/GOOGLE_1_billion_word/heldout"
df_maps = {}
i = 0

In [ ]:
#Emotions
for filename in os.listdir(EMOTIONS_PATH):
    df_maps[i] = read_table(os.path.join(EMOTIONS_PATH,filename), header=None)
    i += 1

In [ ]:
def preprocessing(df):
    df = df.dropna()
    df = df.applymap(lambda x: x.rstrip())
    df = df.applymap(lambda x: x.lstrip())
    df = df.applymap(lambda x: x[0].title() + x[1: ])
    df = df.applymap(lambda x: x + "." if x[-1] != "." else "")
    return df

In [ ]:
def get_utteranes(df, utterances):
    for column in df.columns:
        utterances += list(df[column].unique())
    return utterances


In [ ]:
@autojit
def parralel(pre_data):
    data = []
    l = len(pre_data)
    for i in prange(l):
        data.append(preprocessing(pre_data[i]))

In [ ]:
pre_data = list(df_maps.values())

In [ ]:
start = time.time()
parralel(pre_data)
stop = time.time()
print(stop - start)

In [ ]:
start = time.time()
data=[]
for d in pre_data:
    data.append(preprocessing(d))
stop = time.time()
print(stop - start)

In [ ]:
len(data)

In [ ]:
start = time.time()
data=[]
data = Parallel(n_jobs=-1)(delayed(preprocessing)(d) for d in pre_data)
stop = time.time()
print(stop - start)

In [ ]:
utterances = []
for df in data:
    utterances += get_utteranes(df, utterances)

In [ ]:
shuffle(utterances)

In [ ]:
british_gloss = read_csv("british-english.csv", header=None)
usa_gloss = read_csv("american-english.csv", header=None)

In [ ]:
british_gloss.columns = ["a"]
usa_gloss.columns = ["a"]
words = list(british_gloss.a.unique()) + list(",./';[]<>?:{}!@#$%^&*()_+-=") + list(usa_gloss.a.unique())
words = list(set(words))

In [ ]:
def gg(utt, gloss):
    oov = False
#     try:
    tknzr1 = word_tokenize(utt)
    for token in tknzr1:
        if token.lower() not in gloss:
            oov = True
            break
    if not oov:
        return utt

In [ ]:
def get_valid_utt(data, gloss):
    valid_utt = []
    for utt in data:
        oov = False
        try:
            tknzr1 = word_tokenize(utt)
            for token in tknzr1:
                if token.lower() not in gloss:
                    oov = True
                    break
            if not oov:
                valid_utt.append(utt)
        except:
            print(utt)
    return valid_utt

In [ ]:
len(utterances)//600000

In [ ]:
start = time.time()
valid = get_valid_utt(utterances[:5000], words)
stop = time.time()
print(stop - start)

In [ ]:
len(utterances)// 100000

In [ ]:
ddd = [utterances[i * 50: (i + 1) * 50] for i in range(2 * len(utterances)// 10000)]

In [ ]:
start = time.time()
data=[]
validd = Parallel(n_jobs=-1)(delayed(get_valid_utt)(utt, words) for utt in ddd)
stop = time.time()
print(stop - start)

In [ ]:
valid = []
for el in validd:
    valid += el

In [ ]:
len(valid)